## Basic cluster creation
* The following code will connect to the Dask Gateway server, use jupyterhub OAuth to authenticate and ask the server for a new cluster.
* This willl trigger the server to start a Scheduler pod in OKD, which will be exposed via Traefik. 
* To connect to the scheduler, use the public route: tls://dask-gateway-tls.fnal.gov:443
* This route is static for now. Functionality is being built to expose it dynamically

In [ ]:
import logging
import os
import re
import pprint
import time
import socket
from lpcdaskgateway import LPCGateway

gateway = LPCGateway()
cluster = gateway.new_cluster()

## Staging and interactive worker launching
* In order to have hybrid Dask clusters, we have added a mode of operation to Dask Gateway Server in which external Dask Singularity workers join a containerized Kubernetes scheduler on the fly. 
* The ideal and final goal is to submit these into the grid as batch jobs. This is an intermediate step towards that.
* The following code will use the Dask Gateway API to interact with the cluster, obtain information for the worker to properly connect and authenticate, and finally join the scheduler in a 1:1 hybrid cluster.
* To connect to the scheduler, use the public route: tls://dask-gateway-tls.fnal.gov:443

In [ ]:
cluster_name = cluster.name
print("Staging a static worker for cluster: "+cluster_name)

import pwd

def prepareWorkerArea(cluster):
    username = pwd.getpwuid( os.getuid() )[ 0 ]
    security = cluster.security
    tmproot = f"/uscmst1b_scratch/lpc1/3DayLifetime/{username}/{cluster_name}"
    credentials_dir = f"{tmproot}/dask-credentials"
    worker_space_dir = f"{tmproot}/dask-worker-space"
    image_name = f"/cvmfs/unpacked.cern.ch/registry.hub.docker.com/coffeateam/coffea-dask-cc7-gateway:0.7.12-fastjet-3.3.4.0rc9-g8a990fa"
    worker_name = "dask-gateway-localworker-"+socket.gethostname()
    print("-- Creating base directory and copying credentials")
    os.makedirs(tmproot, exist_ok=True)
    os.makedirs(credentials_dir, exist_ok=True)
    os.makedirs(worker_space_dir, exist_ok=True)

    with open(f"{credentials_dir}/dask.crt", 'w') as f:
        f.write(security.tls_cert)
    with open(f"{credentials_dir}/dask.pem", 'w') as f:
        f.write(security.tls_key)
    with open(f"{credentials_dir}/api-token", 'w') as f:
        f.write(os.environ['JUPYTERHUB_API_TOKEN'])
        
    # Prepare worker.env
    print("-- Writing worker environment file (not supported on batch workers)")
    env = """DASK_GATEWAY_WORKER_NAME="""+worker_name+"""
DASK_GATEWAY_API_URL="https://dask-gateway-api.fnal.gov/api"
DASK_GATEWAY_CLUSTER_NAME="""+cluster_name+"""
DASK_GATEWAY_API_TOKEN=/etc/dask-credentials/api-token
DASK_DISTRIBUTED__COMM__TLS__CA_FILE=/etc/dask-credentials/dask.crt
DASK_DISTRIBUTED__COMM__TLS__SCHEDULER__CERT=/etc/dask-credentials/dask.crt
DASK_DISTRIBUTED__COMM__TLS__SCHEDULER__KEY=/etc/dask-credentials/dask.pem
DASK_DISTRIBUTED__COMM__TLS__WORKER__CERT=/etc/dask-credentials/dask.crt
DASK_DISTRIBUTED__COMM__TLS__WORKER__KEY=/etc/dask-credentials/dask.pem
DASK_DISTRIBUTED__LOGGING__DISTRIBUTED=debug"""
    
    with open(f"{tmproot}/worker.env", 'w+') as f:
        f.writelines(env)
        
    # Prepare singularity command
    print("-- Writing singularity exec command")
    sing = """#!/bin/bash
singularity exec --env-file worker.env  --bind ${PWD}/dask-worker-space:/srv/dask-worker-space  --bind ${PWD}/dask-credentials:/etc/dask-credentials /cvmfs/unpacked.cern.ch/registry.hub.docker.com/coffeateam/coffea-dask-cc7-gateway:latest dask-worker --name """+worker_name+""" --worker-port 10000:10070 --no-nanny --no-dashboard --local-directory /srv --nthreads 2 --nprocs 1 tls://dask-gateway-tls.fnal.gov:443"""
    
    with open(f"{tmproot}/start.sh", 'w+') as f:
        f.writelines(sing)
    os.chmod(f"{tmproot}/start.sh", 0o775)
    return(tmproot)
        
tmproot = prepareWorkerArea(cluster)
print("Done")
print("Attention: Manual steps ahead!")
print("-- Login to a cmslpc interactive node capable of running Singularity")
print("-- `cd` to "+tmproot)
print("-- and finally, run the `start.sh` script")
print("The script will start a Singularity container running a remote dask-gateway worker,")
print("which will join your scheduler and be ready to run Dask analysis")

In [ ]:
cluster

In [ ]:
print(type(cluster))
print(cluster.scheduler_info)

In [ ]:
# Obtain a client for connecting to your cluster scheduler
# Your cluster should be ready to take requests
#cluster = gateway.connect(cluster_name)
client = cluster.get_client()
client

In [ ]:
# Run computations on the cluster
# At this point you should be able to use normal dask methods to do work.
# For example, here we take the mean of a random array.
import dask.array as da
a = da.random.normal(size=(1000, 1000), chunks=(500, 500))
a.mean().compute()

In [ ]:
# Only run this cell to remove/shutdown the cluster
cluster.shutdown()